### Use XGBoost to train toxic/non-toxic data

#### 1.Import related packages

In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

#### 2.Import data

In [3]:
toxic_df = pd.read_csv('/home/liujin/Offtarget_drugsafety/offtarget_application/Toxicity_prediction/toxic_predict_data/toxic_offtarget_profile.csv')
not_toxic_df = pd.read_csv('/home/liujin/Offtarget_drugsafety/offtarget_application/Toxicity_prediction/toxic_predict_data/nontoxic_offtarget_profile.csv')
toxic_df['label'] = 1
not_toxic_df['label'] = 0

df = pd.concat([toxic_df, not_toxic_df], axis=0)
print(toxic_df.shape, not_toxic_df.shape, df.shape)

data_df = df.drop(['smiles','label'], axis=1)
print(data_df.shape)

#### 数据集划分
train_x, test_x, train_y, test_y = train_test_split(data_df, df['label'], test_size=0.2, random_state=999)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)
print(data_df.shape)
print(len(df['label']))

(877, 244) (1229, 244) (2106, 244)
(2106, 242)
(1684, 242) (422, 242) (1684,) (422,)
(2106, 242)
2106


#### 3.Hyperparameter search

In [3]:
import numpy as np
import os
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
import joblib
from sklearn.ensemble import RandomForestClassifier
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [7]:
def objective(trial,X,y):

    X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.1,random_state=999) 
    

    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000,step=50),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'objective': 'binary:logistic',
        'random_state': 2023
    }

    dt_clf = xgb(**param)
    
    dt_clf.fit(X_train, y_train)
    pred_dt = dt_clf.predict(X_val) 
    
    proba = dt_clf.predict_proba(X_val)
    score = roc_auc_score(y_val, proba[:,1])

    return score
    

In [8]:
study = optuna.create_study(direction="maximize", study_name="XGBoost Classifier")
func = lambda trial: objective(trial, train_x, train_y)
study.optimize(func, n_trials=1000) 

[I 2023-09-15 03:11:57,832] A new study created in memory with name: XGBoost Classifier
<ipython-input-7-2d8af442cae7>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
[I 2023-09-15 03:12:00,316] Trial 0 finished with value: 0.9039021549213745 and parameters: {'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.0013453365039547275, 'subsample': 0.5304651327596935, 'colsample_bytree': 0.931249712504733, 'gamma': 1.441727782431127, 'reg_alpha': 0.7211588004238467, 'reg_lambda': 0.2907850779152277, 'min_child_weight': 1}. Best is trial 0 with value: 0.9039021549213745.
<ipython-input-7-2d8af442cae7>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag

In [1]:
print(f"\tBest value (auc): {study.best_value:.5f}")
print(f"\tBest params:")
for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

#### 4.Conduct five-fold cross-training based on the optimum hyperparameter

In [4]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, balanced_accuracy_score, matthews_corrcoef
from xgboost import XGBClassifier as xgb

kf = KFold(n_splits=5, random_state=999, shuffle=True)

train_x, test_x, train_y, test_y = train_test_split(data_df, df['label'], test_size=0.2, random_state=999)

# model = xgb(**study.best_params)
model = xgb(n_estimators=850,
            max_depth=4,
            learning_rate=0.099,
            subsample=0.530,
            colsample_bytree=0.882,
            gamma=7.091,
            reg_alpha=1.879,
            reg_lambda=9.272,
            min_child_weight=7,
            random_state=2023)


acc_list = []
auc_list = []
f1_list = []
bacc_list = []
mcc_list = []

for train_index,val_index in kf.split(train_x):
    X_train, X_val = train_x.iloc[train_index], train_x.iloc[val_index]
    y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(test_x)
    y_prob = model.predict_proba(test_x)

    acc_list.append(accuracy_score(test_y, y_pred))
    auc_list.append(roc_auc_score(test_y, y_prob[:,1]))
    f1_list.append(f1_score(test_y, y_pred, average='binary'))
    bacc_list.append(balanced_accuracy_score(test_y, y_pred))
    mcc_list.append(matthews_corrcoef(test_y, y_pred))

print('accuracy_score:', np.mean(acc_list), np.std(acc_list))
print('roc_auc_score:', np.mean(auc_list), np.std(auc_list))
print('f1_score:', np.mean(f1_list), np.std(f1_list))
print('balanced_accuracy_score:', np.mean(bacc_list), np.std(bacc_list))
print('matthews_corrcoef:', np.mean(mcc_list), np.std(mcc_list))

accuracy_score: 0.8094786729857819 0.008687347289015808
roc_auc_score: 0.904620566001163 0.0029184633611036015
f1_score: 0.7537069147481992 0.01009569877297078
balanced_accuracy_score: 0.8071864702461717 0.008358853664366825
matthews_corrcoef: 0.6018335869584328 0.01705029776975143
